In [2]:
import logging
import asyncio
import contextlib
from imprint.nb_util import setup_nb
setup_nb()
from confirm.adagrid.backend import backup_daemon

    
class DB:
    calls = 0
    def backup(self):
        self.calls += 1

db = DB()
async with backup_daemon(db, backup_interval=1):
    print("Hello world")
    await asyncio.sleep(3)
assert(db.calls == 3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[worker_id=None] 2023-03-13 14:40:31,280 - imprint - DEBUG 
Enabling 64-bit floats in JAX.
Hello world
[worker_id=None] 2023-03-13 14:40:32,285 - confirm.adagrid.backend - INFO 
Backing up database
[worker_id=None] 2023-03-13 14:40:32,286 - confirm.adagrid.backend - INFO 
Backup complete
[worker_id=None] 2023-03-13 14:40:33,287 - confirm.adagrid.backend - INFO 
Backing up database
[worker_id=None] 2023-03-13 14:40:33,287 - confirm.adagrid.backend - INFO 
Backup complete
[worker_id=None] 2023-03-13 14:40:34,285 - confirm.adagrid.backend - INFO 
Backing up database
[worker_id=None] 2023-03-13 14:40:34,286 - confirm.adagrid.backend - INFO 
Backup complete
